In [1]:
import os
import sys

# In case it is not installed as a package
# the project folder needs to be added to
# the path

# Change the path to the project folder
project_path = os.path.expanduser(os.path.join("~", "Documents", "ResultExtractor"))
if project_path not in sys.path:
    sys.path.append(project_path)

# Operations

The ResultExtractor package offers the **operations** module to investigate the runs for different tasks and flows according to different criteria. 
Each function (operation) needs as input a DataFrame with tasks as rows, flows as columns and their corresponding runs as entries (*The input DataFrame can be obtained by using the ResultExtractor with different filters according to your needs*).

Implemented in the module as of right now are the following operations:

* Calculate the average value of all runs for the different flow and task combinations, given an evaluation measure.
* Find the best run for different task and flow combinations, given an evaluation measure.
* Calculate the fraction of runs using RandomSearch that fall into the best found minima region, within a certain threshold, for a given measure for the task and flow combinations.

In [2]:
from src.util import get_flow_ids
from src.result_extractor import ResultExtractor

# Example results for mlr.classif.xgboost_5
flow_ids = get_flow_ids('mlr.classif.xgboost_5')
results = ResultExtractor(*flow_ids).results

## Investigating results based on a given measure

The **get_tasks_by_measure** function, generates results by getting the runs if any for a task and flow combination and averaging their results on the **given evaluation measure**, this is done for all combinations. The function returns a DataFrame similiar to the input, however the entries are not run ids anymore, but the averaged values of all run predictions.

In [3]:
from pprint import pprint
from src.operations import get_tasks_by_measure

averaged_results = get_tasks_by_measure(
    results,
    evaluation_measure='predictive_accuracy'
)
pprint(averaged_results)

           5963
272    0.730000
282    0.735955
3917   0.853016
9983   0.604286
34536  0.914947


## Investigating results based on the best value

The **get_tasks_by_best_score** function, generates results by getting all the runs for a task and flow combination and selecting the one that has the best value for the **given evaluation measure**, this is done for all combinations. If the best value is the lowest value, the *order* argument should be *'descending'*, otherwise*'ascending'*.

The function returns a DataFrame similiar to the input, however, the entries contain the predictive measure value of the best run.

In [4]:
from pprint import pprint
from src.operations import get_tasks_by_best_score

best_results = get_tasks_by_best_score(
    results,
    order='increasing',
    measure='predictive_accuracy'
)
pprint(best_results)

           5963
272    0.730000
282    0.831461
3917   0.864391
9983   0.710881
34536  0.977127


## Investigating results based on the minima region

The **get_tasks_by_minima_region** function, generates results by getting all the runs for a task and flow combination that make use of RandomSearch and calculates the fraction of runs that reach the minima region (the best minima found so far in the results) within a certain threshold. This is done for all combinations. The function makes use of **get_tasks_by_best_score** so that it can find the best minima.

The function has the same arguments as **get_tasks_by_best_score** (explained above), a *threshold* argument which represents how much we are willing to consider as the minima region, a *flow_ids* argument which represents the restrictions in flows and *task_restrictions* which represents the restrictions in the tasks considered. 

**If given, the flow_ids and task_restrictions arguments must match the flows considered in the DataFrame given as input.**

It returns a DataFrame similiar to the input, where the entries are not run ids anymore but the fraction of runs that make use of RandomSearch and reach the minima region.

In [5]:
from pprint import pprint
from src.operations import get_tasks_by_minima_region

region_results = get_tasks_by_minima_region(
    results,
    flow_ids,
    task_restrictions=None,
    order='increasing',
    measure='predictive_accuracy',
    threshold=0.05,
    detailed='No'
)
pprint(region_results)

     5963
272  1.00
282  0.75
